# Azure AI Agent service - Function calling

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import glob
import json
import ipyplot
import matplotlib.pyplot as plt
import os
import requests
import sys
import time
import yfinance as yf

from azure.ai.agents import AgentsClient 
from azure.ai.agents.models import (
    FunctionTool,
    ListSortOrder,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolOutput,
)
from azure.identity import DefaultAzureCredential
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI
from PIL import Image
from typing import Any, Callable, Set, Dict, List, Optional
from user_functions import user_functions, azuremaps_weather

In [2]:
load_dotenv("azure.env")

True

## 1. Agent definition - Weather forecasts

In [3]:
endpoint = os.getenv("PROJECT_ENDPOINT")
credential = DefaultAzureCredential()

project_client = AgentsClient(endpoint=endpoint, credential=credential)

### Testing the Azure Maps Weather function

We will use the Azure Maps service to provide weather information.

In [4]:
weather = azuremaps_weather("Toronto")
json.loads(weather)

{'weather_data': {'results': [{'dateTime': '2025-06-10T05:17:00-04:00',
    'phrase': 'Mostly clear',
    'iconCode': 34,
    'hasPrecipitation': False,
    'isDayTime': False,
    'temperature': {'value': 12.8, 'unit': 'C', 'unitType': 17},
    'realFeelTemperature': {'value': 12.3, 'unit': 'C', 'unitType': 17},
    'realFeelTemperatureShade': {'value': 12.3, 'unit': 'C', 'unitType': 17},
    'relativeHumidity': 100,
    'dewPoint': {'value': 12.8, 'unit': 'C', 'unitType': 17},
    'wind': {'direction': {'degrees': 248.0, 'localizedDescription': 'WSW'},
     'speed': {'value': 9.3, 'unit': 'km/h', 'unitType': 7}},
    'windGust': {'speed': {'value': 9.3, 'unit': 'km/h', 'unitType': 7}},
    'uvIndex': 0,
    'uvIndexPhrase': 'Low',
    'visibility': {'value': 9.7, 'unit': 'km', 'unitType': 6},
    'obstructionsToVisibility': 'F',
    'cloudCover': 11,
    'ceiling': {'value': 12192.0, 'unit': 'm', 'unitType': 5},
    'pressure': {'value': 1006.4, 'unit': 'mb', 'unitType': 14},
    'pr

### Agent

In [5]:
model = "gpt-4o-mini"
name = "gpt-4o-mini-weather-agent"
instructions = "You are a weather bot. Use the provided functions to help answer questions."

functions = FunctionTool(user_functions)

# Create an agent and run user's request with function calls
agent = project_client.create_agent(
    model="gpt-4o",
    name=name,
    instructions=instructions,
    tools=functions.definitions,
    description="Weather agent using Azure Maps",
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_AtWIKCQ5eNmeipBttg6b5oQa


In [6]:
id1 = agent.id

### Tests

In [7]:
prompt = "Hello, generate a full report for the weather in Paris today"

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_ZRTUsFudJZ5nq5i7JCjgAVJJ
Created message ID = msg_IzyEiABQ7inz4ttITuBgZGEz


In [8]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_zF7hxjarfSxhPIn8G8NEwn76


In [9]:
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        if tool_outputs:
            project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

Executing tool call: {'id': 'call_HJz5F6o1mccMoVABO4P0QG0U', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Paris"}'}}


In [10]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-10 09:26:42+0000
End     : 2025-06-10 09:26:47+0000
Elapsed : 0:00:05  (5.00 seconds)


{'prompt_tokens': 1494, 'completion_tokens': 399, 'total_tokens': 1893, 'prompt_token_details': {'cached_tokens': 0}}

In [11]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

user: Hello, generate a full report for the weather in Paris today
assistant: Here is the detailed weather report for Paris today:

- **Date & Time:** June 10, 2025 (current observation at 11:16 AM)
- **General Description:** Sunny
- **Temperature:** 20.0°C
  - **Feels Like:** 24.1°C
  - **Feels Like in Shade:** 18.5°C
  - **Apparent Temperature:** 19.4°C
  - **Wind Chill:** 20.0°C
  - **Wet Bulb Temperature:** 14.9°C

### Wind
- **Direction:** Northwest (NW), 315°
- **Speed:** 12.0 km/h
- **Wind Gusts:** Up to 18.4 km/h

### UV Index
- **Level:** 8 (Very High)
  - Caution: Protection against the sun is highly recommended.

### Humidity
- **Relative Humidity:** 58%
- **Dew Point:** 11.5°C

### Visibility
- **Distance:** 24.1 km
- **Obstructions:** None observed

### Cloud Cover
- **Cloud Coverage:** 10%
- **Ceiling (Cloud Base):** 12,192 meters

### Pressure
- **Atmospheric Pressure:** 1019 mb
- **Tendency:** Steady

### Precipitation
- **Rainfall Summary:**
  - Past Hour: 0.0 mm
  - P

### Other examples

In [12]:
prompt = "Hello, generate a full report for the weather in Mexico City today. Print the results in English, Spanish and French."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_Lx6tXOUlXHhafoMBAmNkBazQ
Created message ID = msg_8j3DkuSqLnAc41YjcJN84Okl


In [13]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_5ERWjISUJS3HNa0xvfyI8qHJ


In [14]:
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        if tool_outputs:
            project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

Executing tool call: {'id': 'call_XqXsWOauKZ55ecDmwlFRgSfn', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Mexico City"}'}}


In [15]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-10 09:26:53+0000
End     : 2025-06-10 09:26:59+0000
Elapsed : 0:00:06  (6.00 seconds)


{'prompt_tokens': 1519, 'completion_tokens': 472, 'total_tokens': 1991, 'prompt_token_details': {'cached_tokens': 0}}

In [16]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

user: Hello, generate a full report for the weather in Mexico City today. Print the results in English, Spanish and French.
assistant: ### Weather Report for Mexico City Today

**In English:**
The current weather in Mexico City is: Partially cloudy with 60% cloud cover and no precipitation. The temperature is 17.8°C, with a RealFeel of 17.9°C. The humidity is at 67% and the wind is coming from the west at a speed of 5.6 km/h. Visibility is excellent with 9.7 km, and the UV index is low (0). Atmospheric pressure remains steady at 1026.4 mb. In the past 24 hours, there was 3 mm of rainfall, with a high of 28.9°C and a low of 14.9°C.

**En Español:**
El clima actual en Ciudad de México es: Parcialmente nublado con un 60% de cobertura de nubes y sin precipitación. La temperatura es de 17.8°C, con una Sensación Térmica de 17.9°C. La humedad es del 67% y el viento proviene del oeste a una velocidad de 5.6 km/h. La visibilidad es excelente con 9.7 km, y el índice UV es bajo (0). La presión at

In [17]:
prompt = "Hello, print the UV informations for Miami (Florida) today. Print the date as well of the weather data."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_05XlzCFC5gxcvOkyGSNHXa7W
Created message ID = msg_zRKY5Ux9C0X95YQxXVxssUPU


In [18]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_NdRXZfzZzXqmLTprWlB77CIS


In [19]:
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        if tool_outputs:
            project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

Executing tool call: {'id': 'call_tGQkCeqDzAFMHuLzTv2d5Y2Q', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Miami, Florida"}'}}


In [20]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-10 09:27:07+0000
End     : 2025-06-10 09:27:08+0000
Elapsed : 0:00:01  (1.00 seconds)


{'prompt_tokens': 1519, 'completion_tokens': 73, 'total_tokens': 1592, 'prompt_token_details': {'cached_tokens': 0}}

In [21]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

user: Hello, print the UV informations for Miami (Florida) today. Print the date as well of the weather data.
assistant: The UV information for Miami, Florida is as follows:

- **Date and Time of Weather Data**: June 10, 2025 (05:12 AM local time)
- **UV Index**: 0
- **UV Index Phrase**: Low


In [22]:
prompt = "Generate a weather report for Rome. This report is for a blog publication. Add emojis and adopt a fun style."

# Create thread for communication
thread = project_client.threads.create()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_78iET8OawmvYyfxOeaB4wgqF
Created message ID = msg_OSTVnuAh4pRo79ejCwmOS2KV


In [23]:
run = project_client.runs.create(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_UxGyzm4q1MKKYpsIsmAlrlSy


In [24]:
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        if tool_outputs:
            project_client.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

Executing tool call: {'id': 'call_mlyEB74lJlinxUdfBN2suFjS', 'type': 'function', 'function': {'name': 'azuremaps_weather', 'arguments': '{"query":"Rome"}'}}


In [25]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

run.usage

Start   : 2025-06-10 09:27:13+0000
End     : 2025-06-10 09:27:18+0000
Elapsed : 0:00:05  (5.00 seconds)


{'prompt_tokens': 1517, 'completion_tokens': 394, 'total_tokens': 1911, 'prompt_token_details': {'cached_tokens': 0}}

In [26]:
# Fetch and log all messages
messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

user: Generate a weather report for Rome. This report is for a blog publication. Add emojis and adopt a fun style.
assistant: 🌞 Buongiorno, Rome lovers! Here's your weather dose from the Eternal City! 🇮🇹

Today, Rome is basking in golden sunshine without a single cloud in sight ☀️ (yes, *0% cloud cover* folks, pure sky perfection!). With temperatures sitting at a delightful **27°C (80.6°F)** but feeling like a toasty **31.6°C (88.9°F)**, you're looking at classic summery Rome vibes. 🔥

The West-Southwest breeze is keeping things relaxed at **10.7 km/h (6.6 mph)**, with a few cheeky gusts hitting **20.3 km/h (12.6 mph)**. Perfect to cool you off as you stroll past the Colosseum or indulge in some gelato! 🍦🍃

**UV Index?** A sizzling 9 (!!)—that's "Very High"! 🕶️🧴 Don't forget your sunglasses and sunscreen to stay sun-safe while enjoying Rome’s cobblestone streets and iconic piazzas. 

**Humidity?** Moderate, at **55%**, so it's warm but not sticky—ideal for outdoor adventures, whether y

### Post processing

In [27]:
# List all agents in the project
print("Listing all agents in the project:")
agents = project_client.list_agents()

for agent in agents:
    print(f"Agent ID: {agent.id}, Name: {agent.name}, Model: {agent.model}, Instructions: {agent.instructions}")

Listing all agents in the project:
Agent ID: asst_AtWIKCQ5eNmeipBttg6b5oQa, Name: gpt-4o-mini-weather-agent, Model: gpt-4o, Instructions: You are a weather bot. Use the provided functions to help answer questions.


In [28]:
#project_client.delete_agent(id1)
#print(f"Deleted agent, agent ID: {id1}")